In [ ]:
import numpy as np
import pandas as pd
import cufflinks as cf
cf.go_offline()
import seaborn as sns

# Gestion indicielle

## Gestion benchmarkée

La gestion indicelle est un type de gestion ou l'on doit battre un benchmark ou alors le repliquer le mieux possible. Si on prend un portefeuille $x$ et un benchmark $b$, on appelle l'erreur de réplication ( ou tracking error) l'écart suivant :

$$ \begin{array}{ccc}
e(t) &=& R_x(t)- R_b(t)\\
  &=&(x-b)^\top R(t)
\end{array}$$

On peut alors avoir la variance de la tracking error et la sur/sous performance moyenne :

$$ \begin{array}{ccc}
\mu(x|b) &=& (x-b)^\top \mu\\
\sigma^2(x|b) &=& (x-b)^\top \Sigma (x-b)
\end{array}$$

Ce sont exactement les memes formules que lorsque l'on regarde uniquement le portefeuille $x$ sauf que l'on regarde une difference de portefeuille.

On obtient également une mesure semblable au ratio de sharpe que l'on appelle le ratio d'information :

$$\theta(x|b) = \frac{\mu(x|b) }{\sigma(x|b) }  $$

Si on veut essayer de battre un indice on peut transformer le probleme de markowitz de la façon suiavnte :

\begin{eqnarray*}
x^{\star } &=&\arg \min \frac{1}{2}(x-b)^{\top }\Sigma (x-b) -\color{blue}{\phi \mu^\top (x-b)} \\
&\text{u.c.}&\left\{
\begin{array}{l}
x\in \mathcal{C} \\
\mathbf{1}^{\top }x=1 \\
x\geq \mathbf{0}
\end{array}
\right.  
\end{eqnarray*}

* Le paramètre $\phi$ est l'aversion à la tracking error. 
    *  ce paramètre sert à controler le risque de tracking error.

*  En gestion indicielle on peut egalement vouloir répliquer un indice. Par exemple repliquer le S&P500 mais avec une contrainte sur le nombre d'actif dans le portefeuille : **technique de l'echantillonage**



Plusieurs techniques : 

 * Enlever les actifs les plus petits en poids les uns après les autres tant que la contrainte de TE est respectée


 * Utilisation de la norme $L_1$ : cette norme permet de "sparsifier" le programme de markowitz :
   $$\begin{eqnarray*}
    x^{\star } &=&\arg \min \frac{1}{2}(x-b)^{\top }\Sigma (x-b) + \color{blue}{\lambda \sum_i |x_i| }\\
    &\text{u.c.}&\left\{
    \begin{array}{l}
    x\in \mathcal{C} \\
    \mathbf{1}^{\top }x=1 \\
    x\geq \mathbf{0}
    \end{array}
    \right.  
    \end{eqnarray*}$$
 

* Utilisation de contrainte entière (mixed integer quadratic programming, MIQP) :
    $$\begin{eqnarray*}
    x^{\star } &=&\arg \min \frac{1}{2}(x-b)^{\top }\Sigma (x-b) \\
    &\text{u.c.}&\left\{
    \begin{array}{l}
    x\in \mathcal{C} \\
    \mathbf{1}^{\top }x=1 \\
    x\geq \mathbf{0}\\
    \color{blue}{\sum_i \mathbf{1}\{ x_i>0\} \leq k}
    \end{array}
    \right.  
    \end{eqnarray*}$$

## Réplication d'indice à composantes cachées

Parfois on peut vouloir répliquer un indice ou un fonds dont on ne connait pas les expositions. Typiquement un indice représentatif de stratégie hedge funds.

In [ ]:
data = pd.read_excel("Exercice7TREX.xlsx",parse_dates=True,skiprows=7,header=0,index_col=0).loc["2007":]
hf =  data["HF Index"] 
X = data.drop("HF Index",1)
(hf+1).cumprod().iplot()

On va supposer que le rendement de cet indice s'écrit :

$$R_I(t) = \sum_{i\in \mathcal{I}} \beta_i R_i(t) $$

Ou $\mathcal{I}$ est l'univers d'investissement à priori. Le problème est donc résolu par simple régression linéraire. 

**Exercice** : calculer la stratégie de réplication de l'indice et la tracking error. Comment améliorer ? 

In [ ]:
eonia = pd.read_csv("eonia.csv",sep=";",index_col=0, parse_dates=True,infer_datetime_format=True) 
eonia = eonia.loc[X.index,:]
#calcul du pas de temps
dt = pd.DataFrame(eonia.index[1:].values-eonia.index[:-1].values,columns={"dt"})
#calcul du rendement de eonia, vaut 0 en date 0
r_eonia =  np.array(eonia['EONIA Index'])*( np.hstack((dt["dt"].shift(-1).dt.days/365,1/365)))
r_eonia = np.nan_to_num(r_eonia)

 
track  = pd.DataFrame((hf+1).cumprod())
track["repliquant"] = (X.dot((np.linalg.inv(X.T.dot(X)).dot(X.T.dot(hf))  ))+r_eonia+1).cumprod()
track.plot()
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.legend(fontsize=15)

# Trading trend following and mean reversion

### Le trend following

Le suivi de tendance est une strategie de hedge fund très connu depuis les années 90. Elle repose sur une hypothese de bias de marche : les investisseurs on tendance à acheter le marché quand il monte et le vendre quand il baisse créant un effet de tendance.

* Historiquement on utlisait des croiseement de moyenne mobiles.
* Aujourd'hui on utilise des techniques plus modernes combinant Markowitz, l'optimisation convexe et la régularization.

In [ ]:
import pandas as pd
data = pd.read_csv("data.csv",sep=";",index_col=0,  parse_dates=True,infer_datetime_format=True)

In [ ]:
r = data.pct_change()
mu = r.ewm(halflife=130).mean()*260
sd = r.ewm(halflife=130).std()*260**0.5
w = mu/sd**2
(w.shift().fillna(0)*r).iloc[260:,:].sum(1).cumsum().plot()